In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = '/global/homes/e/ecastori/PostBorn/'

In [ ]:
data = np.loadtxt(path+'FFT_k-1Pk.dat',skiprows=2)

#----------------------#
n         = data[:,0]
Re_c_n    = data[:,1]
Im_c_n    = data[:,2]
Re_nu_n   = data[:,3]
Im_nu_n   = data[:,4]
#----------------------#

c_n  = Re_c_n  + 1j * Im_c_n
nu_n = Re_nu_n + 1j * Im_nu_n

In [ ]:
data = np.loadtxt(path+'Il_nu_t_new1.dat',skiprows=1)

#----------------------#
ell       = data[:,0]
nn        = data[:,1]
tt        = data[:,2]
Re_I      = data[:,3]
Im_I      = data[:,4]
#----------------------#

I_ = Re_I + 1j * Im_I

In [ ]:
data = np.loadtxt(path+'GG_weights.dat',skiprows=2)
t1   = data[:,0]
w1   = data[:,1]

In [ ]:
print(np.unique(tt)-t1)

In [ ]:
#cosmology
h         = 0.6770
omega_cdm = 0.11923
omega_b   = 0.02247
Omega_b   = omega_b/h**2
Omega_cdm = omega_cdm/h**2
Omega_m   = Omega_b+Omega_cdm
ns        = 0.96824
A0        = 2.10732*10**(-9)

#dictionary for class
cosmo_dict={'h': h,
'omega_b' : omega_b,
'omega_cdm': omega_cdm,
'A_s'    : A0,
'n_s'    : ns,
'k_pivot' : 0.05}
print(cosmo_dict)

# speed of light
c      = 299792458./1000. # km/s

#prefactor for Cl_kk computation from Cl_dd
prefac = 1.5*Omega_m*(100.)**2/c**2 #without h

In [ ]:
from classy import Class
cosmo = Class()
cosmo.set(cosmo_dict)
cosmo.compute()
cosmo_b               = cosmo.get_background()

class_z               = cosmo_b['z'][::-1]
class_chi             = cosmo_b['comov. dist.'][::-1]
class_D               = cosmo_b['gr.fac. D'][::-1]#/cosmo_b['gr.fac. D'][-1]

derivParams           = cosmo.get_current_derived_parameters(['z_rec'])
z_cmb                 = derivParams['z_rec']


from scipy.interpolate import interp1d
from scipy.interpolate import InterpolatedUnivariateSpline as ius
chi_z = interp1d(class_z,class_chi*h)
z_chi = interp1d(class_chi*h,class_z)  # Mpc/h
D_chi = interp1d(class_chi*h,class_D)    # growth
D_z   = interp1d(class_z,class_D)
# chi_cmb
chi_cmb = chi_z(z_cmb)



In [ ]:
plt.figure()
print(chi_z(1100),chi_z(z_cmb))

In [ ]:
z = np.linspace(0,5,100)
plt.figure()
plt.plot(class_z,class_D,ls='-')
plt.plot(z,D_z(z),ls='--')
plt.plot(z,1/(1+z),ls='-.')
plt.xlim(0,5)
plt.show()
D_z(0)

In [ ]:
z=np.linspace(0.,10,200)
plt.figure()
plt.loglog(z,chi_z(z))
plt.show()
plt.figure()
plt.plot(z,D_z(z)/D_chi(chi_z(z)))
plt.show()
# plt.figure()
# plt.loglog(z,)
# plt.show()
# plt.figure()
# plt.loglog(z,z_chi(chi_z(z)))
# plt.show()

In [ ]:
D_chi(chi_z(0.1))

In [ ]:
4./np.pi

In [ ]:
def set_kernel(chi_max):
    def kernel(chi):
        return (1.+z_chi(chi))*D_chi(chi)
    return kernel

In [ ]:
W = set_kernel(chi_cmb)

In [ ]:
ell_ = np.unique(ell)
nu_n_= np.unique(nu_n) 
print(nu_n_)
print(ell_)
print(np.unique(tt))
t_=np.unique(tt)
assert(np.allclose(t_,t1))
print(len(nu_n_),len(n),len(c_n),len(t_))
print(len(tt),len(I_))

In [ ]:
# nu=nu_n_

# res=[]
# m = 0
# for ii,ll in enumerate(ell_):
#     print(ii,ll)
#     for t in t_:
#         for r in t_:
#             res+=[np.real(sum(I_[m:m+len(n)][0:-1]*c_n[0:-1]*2.*(chi_cmb*r)**(-nu[0:-1]))+I_[m:m+len(n)][-1]*c_n[-1]*(chi_cmb*r)**(-nu[-1]))]
#         m+=len(n)
# assert(m==len(I_))

In [ ]:
from scipy.integrate import simps, quadrature
from scipy.interpolate import interp1d
m=0
result=[]
for ii,ll in enumerate(ell_):
    print(ii)
    res1=[]
    for nu in nu_n_:
        res2=[]
        for r in t_:
            integrand=(1.-r*t_)/t_*(1.-r)/r*I_[m:m+len(t_)]*W(r*t_*chi_cmb)*W(r*chi_cmb)
            res2+=[np.sum(w1*integrand)]
            
        m+=len(t_)
        integrand = np.asarray(res2)*(t_*chi_cmb)**(-nu)
        res1+=[np.sum(integrand*w1)]
    result+=[np.real(np.sum(res1[0:-1]*c_n[0:-1]*2.)+res1[-1]*c_n[-1])]
assert(m==len(I_))    

In [ ]:
# from scipy.integrate import simps, quadrature
# from scipy.interpolate import interp1d
# m=0
# result=[]
# for ii,ll in enumerate(ell_):
#     res_=[]
#     for t in t_:
        
#         #for r in t_:
#         integrand=W(t*t_*chi_cmb)*W(t*chi_cmb)*(1.-t_*t)/t*(1.-t_)/t_*res[m:m+len(t_)]

            
#         m+=len(t_)
#         #integrand[0] = 0.#+1j*0.

#         res_+=[np.sum(w1*integrand)]#quadrature(integrand,min(t_),max(t_))[0]]
#     #print(res_)
#         # for every n do r integral
#     integrand = np.asarray(res_)

#     if ll==3:
#         plt.plot(t_,integrand,marker='o')
#     #integrand[0] = 0.#+1j*0.
#     result+=[simps(integrand,t_)]#min(t_),max(t_))[0]]
    


In [ ]:
# from scipy.integrate import simps, quadrature
# from scipy.interpolate import interp1d
# m=0
# result=[]
# for ii,ll in enumerate(ell_):
#     res_=[]
#     for t in t_:
        
#         #for r in t_:
#         integrand=W(t*t_*chi_cmb)*(1.-t_*t)*(1.-t_)/t_*res[m:m+len(t_)]

            
#         m+=len(t_)
#         integrand[0] = 0.#+1j*0.
#         #if ll==20:
#         #    plt.semilogx(t_,integrand)
#         integrand = interp1d(t_,integrand,kind='linear')
#         t_long = np.linspace(min(t_),max(t_),500)
#         res_+=[simps(integrand(t_long),t_long)]#quadrature(integrand,min(t_),max(t_))[0]]
#     #print(res_)
#         # for every n do r integral
#     integrand = np.asarray(res_)*W(t_*chi_cmb)/t_

    
#     integrand[0] = 0.#+1j*0.
#     integrand = interp1d(t_,integrand,kind='linear')
#     if ll==3:
#         plt.semilogx(t_long,integrand(t_long))
#     result+=[simps(integrand(t_long),t_long)]#min(t_),max(t_))[0]]
        

In [ ]:
Cl=np.real(np.asarray(result))*4./np.pi**2*prefac**2/4.
print(ell_,Cl)

In [ ]:
plt.figure()
plt.loglog(ell_,(ell_*(ell_+1.))**2*Cl/(2.*np.pi))
plt.xlim(2,500)

In [ ]:
cosmo = Class()

cosmo_dict={'h': h,
'omega_b' : omega_b,
'omega_cdm': omega_cdm,
'A_s'    : A0,
'n_s'    : ns,
'k_pivot' : 0.05}
cosmo_dict['output']='lCl, tCl'
cosmo_dict['l_max_scalars']=2000
cosmo_dict['lensing']='yes'
cosmo_dict['l_switch_limber']=100
cosmo_dict['perturb_sampling_stepsize']=0.01

cosmo.set(cosmo_dict)
cosmo.compute()
cllen= cosmo.lensed_cl(lmax=1000)
ll = cllen['ell']
clpp = cllen['pp']

In [ ]:
data=np.loadtxt(path+'Cell_marko.dat')
cl = data[:,1]

ls = data[:,0]


In [ ]:
plt.figure()
plt.loglog(ell_,Cl,label='us')
plt.loglog(ll,clpp,label='CLASS')
plt.loglog(ls,cl,label='Marko')
plt.xlim(2,1000)
plt.ylim(1e-15,1e-7)
plt.legend()


In [ ]:
Cl

In [ ]:
cosmo = Class()

cosmo_dict={'h': h,
'omega_b' : omega_b,
'omega_cdm': omega_cdm,
'A_s'    : A0,
'n_s'    : ns,
'k_pivot' : 0.05}
cosmo_dict['output']='lCl, tCl, mPk'
cosmo_dict['l_max_scalars']=2000
cosmo_dict['lensing']='yes'
cosmo_dict['l_switch_limber']=1

cosmo.set(cosmo_dict)
cosmo.compute()
cllen= cosmo.lensed_cl(lmax=1000)
ll2 = cllen['ell']
clpp2 = cllen['pp']

cosmo.sigma8()

In [ ]:
plt.figure()#
plt.semilogx(ell_,Cl/np.interp(ell_,ll,clpp), label='our result over CLASS no Limber')
#plt.semilogx(ell_,Cl/np.interp(ell_,ll,clpp)/1.13, label='our result/1.13 over CLASS no Limber')
#plt.semilogx(ell_,np.interp(ell_,ls,cl)/Cl)
#plt.semilogx(ls,cl/np.interp(ls,ll,clpp), label='Markos result over CLASS no Limber')
#plt.axhline(1,color='black')
plt.ylim(0.95,1.05)
plt.grid()
plt.legend()

In [ ]:
import camb
from camb import model, initialpower
print('CAMB version: %s '%camb.__version__)

In [ ]:
x = y = z = np.arange(0.0,5.0,1.0)
np.savetxt('test.out', (x,y,z))
type(ll)

In [ ]:
np.savetxt('class_ouput.out',X=(ll2,clpp,clpp2))



In [ ]:
plt.semilogx(ll,clpp2/clpp-1, label='CLASS Limber over CLASS no Limber')

In [ ]:
#ratio class over camb
ll,clpp,clpp2 = np.loadtxt('class_ouput.out')
c_ell,cl_limber,cl_limber2,cl_camb = np.loadtxt('camb_cls.out')
clpp=np.interp(c_ell,ll,(ll*(ll+1.))**2*clpp/2./np.pi)
clpp2=np.interp(c_ell,ll,(ll*(ll+1.))**2*clpp2/2./np.pi)
plt.semilogx(c_ell,clpp/cl_camb-1, label='CLASS over CAMB no Limber -1')
plt.ylim(-0.1,0.1)
plt.grid()

In [ ]:
def cmb_kernel(chi):
    return D_chi(chi)*(1.+z_chi(chi))*(chi_cmb-chi)/chi_cmb/chi

In [ ]:
data=np.loadtxt('class_pk.dat')
kh = data[:,0]
Pkh = data[:,1]
plt.figure()
plt.loglog(kh,Pkh)

pki=interp1d(kh,Pkh,bounds_error=False, fill_value=0.)

kk = np.logspace(-5,3)
plt.loglog(kk,pki(kk),ls='--')
print(pki(kk))

In [ ]:
chis = np.linspace(50,chi_cmb,1000)

clpp_=[]
for ll in ell_:
    print(ll)
    integrand =[]
    for chi_ in chis:
        k_ = ll/chi_
        pk = pki(k_)/k_**4 #p_psi
        integrand+=[pk*cmb_kernel(chi_)**2/chi_**2]
    clpp_+=[simps(integrand,chis)]

In [ ]:
clpp_=np.array(clpp_)*prefac**2*2.